<a href="https://colab.research.google.com/github/mlvmalvinn/DeltaSquadron-Random-Code/blob/main/ArtNaon_Model_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Locate The Data Path

###Get the Google drive Path Location, with google sign in

In [ ]:
# Google.colab drive integration
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# specify the desired data location to the current working directory
%cd /content/drive/MyDrive/Dataset

In [ ]:
# Keras prepocessing library
! pip install Keras-Preprocessing

In [ ]:
# Get the required Library
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50V2, VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input, MaxPool2D, BatchNormalization, Dropout, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Model
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

#Splitting The Dataset

In [ ]:
# Library to split datasets
! pip install split_folders

In [ ]:
# Getting the Dataset's Folder path and the splitted Data's target location

import splitfolders # or import splitfolders
input_folder = "/content/drive/MyDrive/Dataset/Art/NoDuplicates/" # Change The path depending on your file

# where you want the split datasets saved. one will be created if it does not exist or none is set
output = "/content/drive/MyDrive/Dataset/SplitData" # Change The path depending on your file

# ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.
splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.8, .2))

In [ ]:
# Get the library required
from keras_preprocessing.image import ImageDataGenerator

# Training Data Path and Image gen (Normalization)
TRAINING_DIR = "/content/drive/MyDrive/Dataset/SplitData/train" # Change The path depending on your file
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Validation Data Path and Image gen (Normalization)
VALIDATION_DIR = "/content/drive/MyDrive/Dataset/SplitData/val" # Change The path depending on your file
validation_datagen = ImageDataGenerator(rescale = 1./255)

# Training gen and batching
train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=32
)

# Validation gen and batching
validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=32
)

#Checking the File's in the Directory

In [ ]:
# Just making sure the library to import things from G Drive Is present
from google.colab import drive
import os # Library to manipulate OS  I.E Copying, changing, and or reading things on folders

directory = '/content/drive/MyDrive/Dataset/SplitData/train/' # Change directory depending on your file's path

# List the contents of the directory
# Code to Check wether there is file's in the directory or not (Might be useful to debug)
try:
    files = os.listdir(directory)
    print("Files in directory:", files)
except FileNotFoundError:
    print(f"The directory {directory} does not exist.")

# If the file's extension is not .JPG, Use this code

In [ ]:
from PIL import Image
import os

def check_and_convert_to_jpeg(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)

            try:
                with Image.open(file_path) as img:

                    # Check if the file is in WEBP format
                    if img.format == 'WEBP': # Set this to What extension you want to change to JPEG / JPG

                        # Convert to JPEG
                        new_file_path = os.path.splitext(file_path)[0] + '.jpg'
                        img = img.convert('RGB')  # Convert to RGB to ensure JPEG compatibility
                        img.save(new_file_path, 'JPEG')
                        print(f"Converted {file_path} to {new_file_path}")
                        os.remove(file_path)  # Remove the original WEBP file / the extension that you set

            except (IOError, SyntaxError) as e:
                print(f"Cannot open file: {file_path} ({e})")

# Check and convert files in the training data directory
train_directory = "/content/drive/MyDrive/Dataset/SplitData/train"
check_and_convert_to_jpeg(train_directory)

# Check and convert files in the validation data directory
val_directory = "/content/drive/MyDrive/Dataset/SplitData/val"
check_and_convert_to_jpeg(val_directory)

#Load the Dataset Onto The Train and Val Variable

In [ ]:
# Load training data
train_data = tf.keras.utils.image_dataset_from_directory(
  "/content/drive/MyDrive/Dataset/SplitData/train",
  image_size=(224, 224),
  batch_size=32,
  seed=123
)

# Load validation data
val_data = tf.keras.utils.image_dataset_from_directory(
  "/content/drive/MyDrive/Dataset/SplitData/val",
  image_size=(224, 224),
  batch_size=32,
  seed=123
)

#Model Attempt 1

In [ ]:
#The Machine Learning Model
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (224,224,3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [ ]:
#Compile The Model
model1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#Fit The Data Onto the previously compiled Model
model1.fit(train_data,
          validation_data=val_data,
          epochs=10) #Change The Epoch (Training Attempt / Loop) Depending On your Judgement

#Model Attempt 2